In [1]:
!nvidia-smi

'nvidia-smi' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
import torch 
from torch import nn

In [3]:
torch.device('cpu')

device(type='cpu')

In [4]:
torch.cuda.device('cuda')

In [5]:
torch.cuda.device('cuda:1')

# 查询可用gpu的数量

In [6]:
torch.cuda.device_count()

0

# 这两个函数允许我们在请求GPU不存在的情况下运行代码

In [11]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i),否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}') # 神奇的return
    return torch.device('cpu')
try_gpu()

device(type='cpu')

In [12]:
try_gpu(0) # 0没有>1所以没有进入if 

device(type='cpu')

In [ ]:
try_gpu(2) # 2没有>3，所以返回cpu 好神奇的代码

In [14]:
def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')] # 神奇，为什么列表可以if
try_all_gpus()

[device(type='cpu')]

# 查询张量所在的设备

In [16]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

# 存储在gpu上

In [17]:
X = torch.ones(2,3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

# 在第二个gpu上创建一个随机张量

In [18]:
Y = torch.rand(2,3,device=try_gpu(1))

In [19]:
Y

tensor([[0.9236, 0.3069, 0.1184],
        [0.0582, 0.7430, 0.2430]])

# 要计算 X + Y ，我们需要决定在哪里 （哪个gpu/cpu执行这个操作）

In [22]:
# 必须要在同一个gpu上  ,有历史原因， 性能的考虑 ，移动来移动去性能会差，所以让你做决定
Z = X.cuda(1)
print(X)
print(Z)

AssertionError: Torch not compiled with CUDA enabled

In [23]:
Y+Z

NameError: name 'Z' is not defined

In [ ]:
Z.cuda(1) is Z # 如果Z已经在cuda1上就不会做任何事情，只是返回自己，这也是性能上面的一个考虑

# 神经网络与gpu

In [24]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

In [25]:
net(X)

tensor([[0.1633],
        [0.1633]], grad_fn=<AddmmBackward0>)

In [26]:
net = net.to(device='cuda')

AssertionError: Torch not compiled with CUDA enabled

# 确认模型参数存储在同一个gpu上

In [27]:
net[0].weight.data.device

device(type='cpu')